In [6]:
import numpy
import pickle

folder = "/home/sergio/Projects/ssainz/pytorch_bug/"
file_vec = folder + "vec.pickle"
file_net = folder + "nett.pickle"

vec = pickle.load(open(file_vec, "rb"))


import torch
import torch.nn.functional as F
import numpy as np
import torch.nn as nn

ZONE_SIZE = 21
ACTION_SIZE = 22

class DQNv2(nn.Module):
    def __init__(self):
        super(DQNv2, self).__init__()
        self.linear1 = nn.Linear(ZONE_SIZE * 2, 400)
        self.bn1 = nn.BatchNorm1d(400)
        self.linear4 = nn.Linear(400 + ZONE_SIZE + 1, 300)
        self.bn5 = nn.BatchNorm1d(300)
        self.linear6 = nn.Linear(300, ACTION_SIZE,bias=False)
        self.bn6 = nn.BatchNorm1d(ACTION_SIZE)

    def forward(self, x_original):
        x = x_original[:,0:ZONE_SIZE*2]
        #print(x)
        #print(type(x))
        x = x.view(-1, ZONE_SIZE * 2)


        if np.isnan(np.sum(x.cpu().numpy())):
            print("some values from input are nan")
            exit(0)

        one_hot = x_original[:,ZONE_SIZE*2:-1]
        ts = x_original[:,-1:]

        x_mean = torch.mean(x, dim=1).view(-1,1)
        x_std = torch.std(x, dim=1).view(-1,1)
        x = (x -x_mean) / x_std

        x_p = x
        x = self.linear1(x)

        if np.isnan(np.sum(x.cpu().detach().numpy())):

            print "is NANANNA"
            print("x before linear")
            print(x_p)
            print("x before linear shape")
            print(x_p.shape)
            print("x after linear")
            print(x)
            print(x.shape)
            print(self.linear1)


        if np.isnan(np.sum(x.cpu().detach().numpy())):

            print "x_original"
            print x_original
            print "x_original.detach().cpu().numpy().tolist()"
            print x_original.detach().cpu().numpy().tolist()
            #exit(0)

        x = self.bn1(x)
        # print(outl.shape)
        x = F.sigmoid(x)
        # print("x after 1st layer sigmoid")
        # print(x)
        x = self.bn1(x)
        # print("after first layer")
        # print(x)
        x = torch.cat((x, one_hot, ts), dim=1)
        # print("after concat")
        # print(x)
        x = F.sigmoid((self.linear4(x)))
        x = self.bn5(x)
        # print("After second layer")
        # print(x)
        self.linear6(x)
        return x.view(x.size(0), -1)

In [9]:

net = DQNv2()
net.load_state_dict(torch.load(file_net))
vec_tensor = torch.from_numpy(vec)

x = vec[:,0:ZONE_SIZE*2]
x = x.view(-1, ZONE_SIZE * 2)

x_mean = torch.mean(x, dim=1).view(-1,1)
x_std = torch.std(x, dim=1).view(-1,1)
x = (x -x_mean) / x_std

o = net.linear1(x)
print (o)


# vec_tensor = torch.from_numpy(vec)
# print vec_tensor.shape
# out = net(vec_tensor)


ValueError: Type must be a sub-type of ndarray type